In [1]:
%matplotlib inline

import os.path as osp
from nuscenes.utils.geometry_utils import BoxVisibility
import cv2
from nuscenes import NuScenes

nusc = NuScenes(version='v1.0-trainval', dataroot='/data/datasets/nuscenes', verbose=True)


Loading NuScenes tables for version v1.0-trainval...
Loading nuScenes-panoptic...
32 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
34149 panoptic,
Done loading in 21.184 seconds.
Reverse indexing ...
Done reverse indexing in 6.3 seconds.


In [9]:
for idx, scene in enumerate(nusc.scene):
    # print(scene)
    sample = nusc.sample
    # nusc.list_sample(scene['first_sample_token'])

    channel = 'CAM_FRONT'
    first_sample_rec = nusc.get('sample', scene['first_sample_token'])
    sd_rec = nusc.get('sample_data', first_sample_rec['data'][channel])
    
    # last_sample_rec = nusc.get('sample', scene['last_sample_token'])

    window_name= '{}'.format(scene['name'])
    cv2.namedWindow(window_name)
    cv2.moveWindow(window_name, 0, 0)
    
    has_more_frames = True
    milliseconds = 1000
    while has_more_frames:

        # get annotations, boxes, camera_intrinsic
        impath, boxes, camera_intrinsic = nusc.get_sample_data(sd_rec['token'], box_vis_level=BoxVisibility.ANY)

        im = cv2.imread(impath)

        if not osp.exists(impath):
            raise Exception('Error: Missing image %s' % impath)
        for box in boxes:
            c = nusc.explorer.get_color(box.name)
            # print(box.name, box.label)
            box.render_cv2(im, view=camera_intrinsic, normalize=True, colors=(c, c, c))
            sample_annotation = nusc.get('sample_annotation', box.token)
            print("sample_annotation", sample_annotation)
            instance = nusc.get('instance', sample_annotation['instance_token'])
            print("instance", instance)
            category = nusc.get('category', instance['category_token'])
            print("category", category)
            for attribute_token in sample_annotation['attribute_tokens']:
                attribute = nusc.get('attribute', attribute_token)
                print("attribute", attribute)


        cv2.imshow(window_name, im)
        
        key = cv2.waitKey(milliseconds)
        if key == 32: # if space is pressed, pause.
            key = cv2.waitKey()
        if key == 27: # if ESC is pressed, exit.
            cv2.destroyAllWindows()
            break

        if sd_rec['next'] == '':
            has_more_frames = False
            cv2.destroyAllWindows()
        else:        
            prev_sd_rec = sd_rec
            sd_rec = nusc.get('sample_data', sd_rec['next'])

    break


sample_annotation {'token': 'acce0b7220754600b700257a1de1573d', 'sample_token': 'e93e98b63d3b40209056d129dc53ceee', 'instance_token': '39b6c31c9925492a8a9f1b0ef3b877b6', 'visibility_token': '4', 'attribute_tokens': ['58aa28b1c2a54dc88e169808c07331e3'], 'translation': [1002.464, 632.267, 1.936], 'size': [2.312, 7.516, 3.093], 'rotation': [0.9343569321017062, 0.0, 0.0, -0.35633849558178193], 'prev': '', 'next': '4b1c1acb00c04dfe84e54f64a8381649', 'num_lidar_pts': 171, 'num_radar_pts': 7, 'category_name': 'vehicle.truck'}
instance {'token': '39b6c31c9925492a8a9f1b0ef3b877b6', 'category_token': '6021b5187b924d64be64a702e5570edf', 'nbr_annotations': 40, 'first_annotation_token': 'acce0b7220754600b700257a1de1573d', 'last_annotation_token': 'b5cd977a4d054c31936464f9e6201820'}
category {'token': '6021b5187b924d64be64a702e5570edf', 'name': 'vehicle.truck', 'description': 'Vehicles primarily designed to haul cargo including pick-ups, lorrys, trucks and semi-tractors. Trailers hauled after a semi